In [15]:
import xml.etree.cElementTree as ET # Use cElementTree or lxml if too slow
from collections import defaultdict
import re
import pprint
import os
osmfile = "/Users/user/beijingmap.osm"

### 获取sample.osm数据

In [42]:
samplefile= "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(samplefile,'wb') as output:
    output.write(b'<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write(b'<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(osmfile)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write(b'</osm>')

# 审查数据
### 1. 查看各个标签数量

In [4]:
def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag not in tags:
            tags[elem.tag] = 1
        else:
            tags[elem.tag] += 1
    return tags

In [43]:
count_tags(osmfile)

{'bounds': 1,
 'member': 88666,
 'meta': 1,
 'nd': 481554,
 'node': 390671,
 'note': 1,
 'osm': 1,
 'relation': 4240,
 'remark': 1,
 'tag': 219806,
 'way': 71276}

### 2. 获取属性数量

In [6]:
def count_attrs(filename):
    attrs = {}
    for event, elem in ET.iterparse(filename, events=('start', 'end')):
        if event == 'end':
            for attr in elem.attrib:
                if attr not in attrs:
                    attrs[attr] = 1
                else:
                    attrs[attr] += 1
    return attrs

In [19]:
count_attrs(osmfile)

{'changeset': 466187,
 'generator': 1,
 'id': 466187,
 'k': 219806,
 'lat': 390671,
 'lon': 390671,
 'maxlat': 1,
 'maxlon': 1,
 'minlat': 1,
 'minlon': 1,
 'osm_base': 1,
 'ref': 570220,
 'role': 88666,
 'timestamp': 466187,
 'type': 88666,
 'uid': 466187,
 'user': 466187,
 'v': 219806,
 'version': 466188}

### 3. 查看键的数量

In [17]:
def count_keys(filename):
    keys = {}
    for event, elem in ET.iterparse(filename, events=('start', 'end')):
        if event == 'end':
            key = elem.attrib.get('k')
            if key:
                if key not in keys:
                    keys[key] = 1
                else:
                    keys[key] += 1
    return keys

In [25]:
keys = count_keys(osmfile)
sorted_keys = {k: v for (v, k) in sorted([(value, key) for (key, value) in keys.items()], reverse=True)}
sorted_keys

{'real_ale': 1,
 'official_name:az': 1,
 'maxspeed': 144,
 'gps:vdop': 2,
 'official_name:ay': 1,
 'official_name:ar': 1,
 'official_name:as': 1,
 'official_name:av': 1,
 'code': 4,
 'turn:lanes:forward': 2,
 'official_name:an': 1,
 'official_name:ab': 1,
 'official_name:af': 1,
 'official_name:lez': 1,
 'building:height': 1,
 'name:bat-smg': 1,
 'seamark:type': 1,
 'ref:cn:guobao': 9,
 'official_name:bat-smg': 1,
 'created_by': 95,
 'to': 1355,
 'short_name:ab': 1,
 'crossing:barrier': 1,
 'ref:en': 1,
 'name:zu': 1,
 'official_name:pcd': 1,
 'playground': 1,
 'fax': 7,
 'name:kl': 2,
 'official_name:fi': 1,
 'short_name:zh': 30,
 'alt_name:de': 3,
 'social_facility': 1,
 'official_name:fa': 1,
 'name:als': 1,
 'name:ur': 2,
 'official_name:arz': 1,
 'tiger:county': 1,
 'official_name:fy': 1,
 'name:uk': 4,
 'motor_vehicle': 134,
 'name:ug': 2,
 'addr:street': 958,
 'source:name': 3,
 'contact:facebook': 4,
 'distillery': 1,
 'level': 266,
 'name:zh': 3009,
 'roof:material': 341,
 'of

# 清洗数据

### 1. 审查数据中的街道名称项

In [16]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE) #提取街道名称中最后一个单词
expected = ["Avenue","Center","District","Hutong","Jie","Lu","Qu","Road","Street","Alley","SOHO","Chaoyang"]

#整理街道类型，如果street_name的最后单词不在期望内，就添加street_name进street_types
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:            
            street_types[street_type].add(street_name)
            
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

#提取需要审核的街道名称
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    #使用iterparse逐行解析，当它发现在events里指定的一个事件时，就会返回它发现的事件event和元素elem
    for _, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):  #遍历每行node或way下的每个tag标签
                if is_street_name(tag):  #若tag子标签中的k值符合指定值，则调用函数查看v值最后一个单词是否在期望之内
                    audit_street_type(street_types, tag.attrib['v'])                    
                    
    osm_file.close()  
    pprint.pprint(dict(street_types))
    return street_types

st_types = audit(osmfile)

{u'100016': set([u'Chaoyang Qu, Jiangtai W Rd, \u5609\u6797\u8def\u75321\u53f7\u5609\u6797\u82b1\u56ed71\u53f7\u522b\u5885, \u90ae\u653f\u7f16\u7801: 100016']),
 '100026': set(['Floor 10, Jintaili, Chaoyang Road, Chaoyang, Beijing, 100026']),
 '101': set(['101']),
 u'10\u53f7\u9662\u7535\u5b50\u57ceIT\u4ea7\u4e1a\u56ed107\u697c6\u5c42': set([u'\u9152\u4ed9\u6865\u5317\u8def \u753210\u53f7\u9662\u7535\u5b50\u57ceIT\u4ea7\u4e1a\u56ed107\u697c6\u5c42']),
 u'117\u53f7\u5317': set([u'\u6d77\u6dc0\u533a\u897f\u56db\u73af\u5317\u8def117\u53f7\u5317']),
 u'11\u53f7': set([u'\u5317\u4eac\u5e02\u671d\u9633\u533a\u5317\u4e09\u73af\u4e1c\u8def11\u53f7']),
 u'11\u53f7\u9662': set([u'\u5b66\u9662\u8def\u4e0111\u53f7\u9662']),
 u'125\u53f7': set([u'\u671d\u5185\u5317\u5c0f\u8857\u7532125\u53f7']),
 u'133-2\u53f7': set([u'\u671d\u9633\u95e8\u5185\u5927\u8857133-2\u53f7']),
 u'16\u53f7': set([u'\u5efa\u5916\u5927\u8857\u6c38\u5b89\u91cc\u753216\u53f7']),
 u'16\u53f7\u697c\u5e95\u5546': set([u'\u9b4f\u5

### 2. 完善街道名称

In [17]:
mapping = { "ave.": "Avenue",            
            "district": "District",            
            "Rd,": "Road",
            "Rd.": "Road",
            "road": "Road",
            "St": "Street",
            "Str": "Street",
            "street": "Street",
            "jie": "Jie",
            "Chaoyang,":"Chaoyang",
            "chaoyang":"Chaoyang"
            }
def street_id(street):
    for _, elem in ET.iterparse(osmfile, events=("start",)):
        if elem.tag == "node" or elem.tag == "way": 
            for tag in elem.iter("tag"):
                if tag.attrib['v'] == street:                    
                    return elem.attrib['id']   
                
streets_repair = {}
for _, street in st_types.iteritems():     
    for name in street:        
        if street_type_re.search(name).group() in mapping.keys():              
            better_name = name.replace(street_type_re.search(name).group(), 
                    mapping[street_type_re.search(name).group()])            
            streets_repair[street_id(name)] = better_name
            print name, "->", better_name 

print "\nid : better_name"
streets_repair
list(streets_repair.keys())

guanshuyuan hutong street -> guanshuyuan hutong Street
Yang Shan Rd, -> Yang Shan Road
chaoyang -> Chaoyang
19 Ritan. Bei Lu, Chaoyang district -> 19 Ritan. Bei Lu, Chaoyang District
east chang ave. -> east chang Avenue
Dongzhimen Outer St, Chaoyang, -> Dongzhimen Outer St, Chaoyang
North St -> North Street
Chongwenmen Wai St -> Chongwenmen Wai Street
Tie shu xie jie -> Tie shu xie Jie
W. Dazhalan Str -> W. Dazhalan Street
Xizhaosi Str -> Xizhaosi Street
Guozijian Str -> Guozijian Street
Maliandao Rd. -> Maliandao Road
Jiangtai road -> Jiangtai Road
B2 jiangtai road -> B2 jiangtai Road

id : better_name


['103009751',
 '1493881695',
 '4496966993',
 '2956786949',
 '4509278689',
 '4532011991',
 '4115803689',
 '33458936',
 '136146034',
 '3953202674',
 '2250207174',
 '371497126',
 '4367035893',
 '2034233477',
 '4373139852']

# XML转CSV
### 将osm读成list

In [8]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

OSM_FILE = "/Users/user/beijingmap.osm"

NODES_PATH = "nodes.csv" 
NODE_TAGS_PATH = "nodes_tags.csv" 
WAYS_PATH = "ways.csv" 
WAY_NODES_PATH = "ways_nodes.csv" 
WAY_TAGS_PATH = "ways_tags.csv" 

LOWER = re.compile(r'^([a-z]|_)*$')
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

#SCHEMA = schema.schema

NODE_FIELDS = ['id', 'user', 'uid', 'version', 'lat', 'lon', 'timestamp', 'changeset']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'timestamp', 'changeset']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']
            

#将每个元素转换为正确的格式
def shape_element(element, 
                  node_attr_fields = NODE_FIELDS, 
                  way_attr_fields = WAY_FIELDS,
                  problem_chars = PROBLEMCHARS, 
                  default_tag_type = 'regular'
                 ):
    shape_nodes = {}
    shape_ways = {}
    node_attribs = {}  #存放node的key和value, key作表头,value作内容
    way_attribs = {}
    way_nodes = []  
    tags = []

    if element.tag == 'node':            
        for key in NODE_FIELDS:
            node_attribs[key] = element.attrib[key]
        for child in element:              
            NODE_TAGS ={}
            low = re.match(LOWER,child.attrib['k'])
            colon = re.search(LOWER_COLON,child.attrib['k'])            
            problem = re.search(PROBLEMCHARS,child.attrib['k']) 
            
            if colon:
                type_value = child.attrib['k'].split(':')[0]
                NODE_TAGS['id'] = element.attrib['id']
                NODE_TAGS['key'] = child.attrib['k'][len(type_value)+1:]
                    
                if NODE_TAGS['key'] == "street" and NODE_TAGS['id'] in list(streets_repair.keys()):
                    NODE_TAGS['value'] = streets_repair[NODE_TAGS['id']]#把清洗后的街道名称放进变量
                else:
                    NODE_TAGS['value'] = child.attrib['v']
                    
                NODE_TAGS['type'] = type_value
                tags.append(NODE_TAGS)
            elif low:
                NODE_TAGS['id'] = element.attrib['id']
                NODE_TAGS['key'] = child.attrib['k']
                    
                if NODE_TAGS['key'] == "street" and NODE_TAGS['id'] in list(streets_repair.keys()):
                    NODE_TAGS['value'] = streets_repair[NODE_TAGS['id']]#把清洗后的街道名称放进变量
                else:
                    NODE_TAGS['value'] = child.attrib['v']
                    
                NODE_TAGS['type'] = 'regular'
                tags.append(NODE_TAGS)
            elif problem:
                continue        
        shape_nodes = {'node': node_attribs, 'node_tags': tags}
        return shape_nodes
        
    elif element.tag == 'way':        
        for key in WAY_FIELDS:
            way_attribs[key] = element.attrib[key]
        counter = 0
    
        for child in element:
            WAY_NODE_VALUE ={}
            if child.tag =='nd':
                WAY_NODE_VALUE['id']=element.attrib['id']
                WAY_NODE_VALUE['node_id']=child.attrib['ref']
                WAY_NODE_VALUE['position']= counter
                counter+=1
                way_nodes.append(WAY_NODE_VALUE)
        
            if child.tag == 'tag':
                WAY_TAGS ={}
                low = re.match(LOWER,child.attrib['k'])
                problem = re.search(PROBLEMCHARS,child.attrib['k'])
                colon = re.search(LOWER_COLON,child.attrib['k'])
                if colon:
                    type_value = child.attrib['k'].split(':')[0]
                    WAY_TAGS['id'] = element.attrib['id']
                    WAY_TAGS['key'] = child.attrib['k'][len(type_value)+1:]                    
                        
                    if WAY_TAGS['key'] == "street" and WAY_TAGS['id'] in list(streets_repair.keys()):
                        WAY_TAGS['value'] = streets_repair[WAY_TAGS['id']] #把清洗后的街道名称放进变量
                    else:
                        WAY_TAGS['value'] = child.attrib['v']
                    
                    WAY_TAGS['value'] = child.attrib['v']
                    WAY_TAGS['type'] = type_value
                    tags.append(WAY_TAGS)
                elif low:
                    WAY_TAGS['id'] = element.attrib['id']
                    WAY_TAGS['key'] = child.attrib['k']
                        
                    if WAY_TAGS['key'] == "street" and WAY_TAGS['id'] in list(streets_repair.keys()):
                        WAY_TAGS['value'] = streets_repair[WAY_TAGS['id']]
                    else:
                        WAY_TAGS['value'] = child.attrib['v']
                        
                    WAY_TAGS['value'] = child.attrib['v']
                    WAY_TAGS['type'] = 'regular'
                    tags.append(WAY_TAGS)
                elif problem:
                    continue
        shape_ways = {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
        return shape_ways       

### 查看要存入CSV的格式是否符合标准

In [9]:
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            
for element in get_element(OSM_FILE, tags=('node', 'way')): 
    shape_elements = shape_element(element)    
   
    pprint.pprint(dict(shape_elements))

{'node': {'changeset': '53521955',
          'id': '25248662',
          'lat': '39.9059631',
          'lon': '116.3912480',
          'timestamp': '2017-11-05T09:06:50Z',
          'uid': '554973',
          'user': 'dgitto',
          'version': '90'},
 'node_tags': [{'id': '25248662',
                'key': 'admin_level',
                'type': 'regular',
                'value': '2'},
               {'id': '25248662',
                'key': 'alt_name',
                'type': 'regular',
                'value': u'\u5317\u4eac'},
               {'id': '25248662',
                'key': 'en',
                'type': 'alt_name',
                'value': 'Peking'},
               {'id': '25248662',
                'key': 'eo',
                'type': 'alt_name',
                'value': u'Bej\u011dino'},
               {'id': '25248662',
                'key': 'fr',
                'type': 'alt_name',
                'value': 'Beijing'},
               {'id': '25248662',
            

               {'id': '25248662',
                'key': 'tr',
                'type': 'name',
                'value': 'Pekin'},
               {'id': '25248662',
                'key': 'tt',
                'type': 'name',
                'value': u'\u041f\u0435\u043a\u0438\u043d'},
               {'id': '25248662',
                'key': 'ug',
                'type': 'name',
                'value': u'\u0628\u06d0\u064a\u062c\u0649\u06ad \u0634\u06d5\u06be\u0649\u0631\u0649'},
               {'id': '25248662',
                'key': 'uk',
                'type': 'name',
                'value': u'\u041f\u0435\u043a\u0456\u043d'},
               {'id': '25248662',
                'key': 'ur',
                'type': 'name',
                'value': u'\u0628\u06cc\u062c\u0646\u06af'},
               {'id': '25248662',
                'key': 'uz',
                'type': 'name',
                'value': 'Pekin'},
               {'id': '25248662',
                'key': 'vi',
          

          'timestamp': '2017-09-19T17:55:05Z',
          'uid': '4814295',
          'user': u'\u0410\u043b\u0435\u043a\u0441 \u041c\u043e\u043a',
          'version': '20'},
 'node_tags': []}
{'node': {'changeset': '52188536',
          'id': '25585128',
          'lat': '39.9029622',
          'lon': '116.3794549',
          'timestamp': '2017-09-19T17:55:05Z',
          'uid': '4814295',
          'user': u'\u0410\u043b\u0435\u043a\u0441 \u041c\u043e\u043a',
          'version': '7'},
 'node_tags': []}
{'node': {'changeset': '52188536',
          'id': '25585130',
          'lat': '39.9029542',
          'lon': '116.3809818',
          'timestamp': '2017-09-19T17:55:05Z',
          'uid': '4814295',
          'user': u'\u0410\u043b\u0435\u043a\u0441 \u041c\u043e\u043a',
          'version': '9'},
 'node_tags': []}
{'node': {'changeset': '52188536',
          'id': '25585132',
          'lat': '39.9029552',
          'lon': '116.3816415',
          'timestamp': '2017-09-19T17:55:05Z'

{'node': {'changeset': '12466867',
          'id': '27484999',
          'lat': '39.9062558',
          'lon': '116.3107059',
          'timestamp': '2012-07-24T13:39:49Z',
          'uid': '376715',
          'user': 'R438',
          'version': '14'},
 'node_tags': [{'id': '27484999',
                'key': 'highway',
                'type': 'regular',
                'value': 'traffic_signals'}]}
{'node': {'changeset': '12415788',
          'id': '27485058',
          'lat': '39.9096329',
          'lon': '116.3227946',
          'timestamp': '2012-07-21T17:15:58Z',
          'uid': '376715',
          'user': 'R438',
          'version': '3'},
 'node_tags': []}
{'node': {'changeset': '11559190',
          'id': '27486094',
          'lat': '39.9662639',
          'lon': '116.3639092',
          'timestamp': '2012-05-10T14:04:06Z',
          'uid': '376715',
          'user': 'R438',
          'version': '7'},
 'node_tags': [{'id': '27486094',
                'key': 'highway',
     

{'node': {'changeset': '117345',
          'id': '30993525',
          'lat': '39.9890272',
          'lon': '116.3988990',
          'timestamp': '2007-06-28T04:32:37Z',
          'uid': '9112',
          'user': 'donsmith',
          'version': '1'},
 'node_tags': []}
{'node': {'changeset': '117345',
          'id': '30993526',
          'lat': '39.9888957',
          'lon': '116.3987188',
          'timestamp': '2007-06-28T04:32:37Z',
          'uid': '9112',
          'user': 'donsmith',
          'version': '1'},
 'node_tags': []}
{'node': {'changeset': '117345',
          'id': '30993527',
          'lat': '39.9887576',
          'lon': '116.3983755',
          'timestamp': '2007-06-28T04:32:37Z',
          'uid': '9112',
          'user': 'donsmith',
          'version': '1'},
 'node_tags': []}
{'node': {'changeset': '117345',
          'id': '30993528',
          'lat': '39.9887181',
          'lon': '116.3978777',
          'timestamp': '2007-06-28T04:32:37Z',
          'uid':

...

          'version': '2'},
 'node_tags': []}
{'node': {'changeset': '30955428',
          'id': '274730919',
          'lat': '39.9928931',
          'lon': '116.5425817',
          'timestamp': '2015-05-10T06:55:28Z',
          'uid': '2461407',
          'user': 'Kartografiertier',
          'version': '2'},
 'node_tags': []}
{'node': {'changeset': '30955428',
          'id': '274730925',
          'lat': '39.9920842',
          'lon': '116.5427765',
          'timestamp': '2015-05-10T06:55:28Z',
          'uid': '2461407',
          'user': 'Kartografiertier',
          'version': '2'},
 'node_tags': []}
{'node': {'changeset': '30955428',
          'id': '274730928',
          'lat': '39.9912382',
          'lon': '116.5429228',
          'timestamp': '2015-05-10T06:55:28Z',
          'uid': '2461407',
          'user': 'Kartografiertier',
          'version': '2'},
 'node_tags': []}
{'node': {'changeset': '30955428',
          'id': '274730930',
          'lat': '39.9902930',
     

          'version': '4'},
 'node_tags': []}
{'node': {'changeset': '26584653',
          'id': '274731185',
          'lat': '39.9853331',
          'lon': '116.5119736',
          'timestamp': '2014-11-06T02:42:30Z',
          'uid': '397326',
          'user': 'trolleway',
          'version': '4'},
 'node_tags': []}
{'node': {'changeset': '26584653',
          'id': '274731190',
          'lat': '39.9856009',
          'lon': '116.5117133',
          'timestamp': '2014-11-06T02:42:30Z',
          'uid': '397326',
          'user': 'trolleway',
          'version': '4'},
 'node_tags': []}
{'node': {'changeset': '26584542',
          'id': '274731194',
          'lat': '39.9863386',
          'lon': '116.5111499',
          'timestamp': '2014-11-06T02:28:33Z',
          'uid': '397326',
          'user': 'trolleway',
          'version': '4'},
 'node_tags': []}
{'node': {'changeset': '30955845',
          'id': '274731199',
          'lat': '39.9869599',
          'lon': '116.5107066

          'version': '3'},
 'node_tags': []}
{'node': {'changeset': '12392102',
          'id': '274737242',
          'lat': '39.9828017',
          'lon': '116.5143300',
          'timestamp': '2012-07-20T21:11:07Z',
          'uid': '722137',
          'user': 'OSMF Redaction Account',
          'version': '3'},
 'node_tags': []}
{'node': {'changeset': '12392102',
          'id': '274737253',
          'lat': '39.9829283',
          'lon': '116.5139621',
          'timestamp': '2012-07-20T21:11:07Z',
          'uid': '722137',
          'user': 'OSMF Redaction Account',
          'version': '3'},
 'node_tags': []}
{'node': {'changeset': '12392102',
          'id': '274737267',
          'lat': '39.9832341',
          'lon': '116.5132689',
          'timestamp': '2012-07-20T21:11:07Z',
          'uid': '722137',
          'user': 'OSMF Redaction Account',
          'version': '3'},
 'node_tags': []}
{'node': {'changeset': '12392102',
          'id': '274737278',
          'lat': '39.

          'version': '2'},
 'node_tags': []}
{'node': {'changeset': '30955428',
          'id': '274737525',
          'lat': '39.9778002',
          'lon': '116.5205957',
          'timestamp': '2015-05-10T06:55:27Z',
          'uid': '2461407',
          'user': 'Kartografiertier',
          'version': '2'},
 'node_tags': []}
{'node': {'changeset': '30955428',
          'id': '274737529',
          'lat': '39.9776434',
          'lon': '116.5211928',
          'timestamp': '2015-05-10T06:55:27Z',
          'uid': '2461407',
          'user': 'Kartografiertier',
          'version': '2'},
 'node_tags': []}
{'node': {'changeset': '30955428',
          'id': '274737532',
          'lat': '39.9773775',
          'lon': '116.5224275',
          'timestamp': '2015-05-10T06:55:27Z',
          'uid': '2461407',
          'user': 'Kartografiertier',
          'version': '2'},
 'node_tags': []}
{'node': {'changeset': '30955428',
          'id': '274737537',
          'lat': '39.9771957',
     

NameError: global name 'streets_repair' is not defined

### 将清洗后的数据存入CSV

In [18]:
OSM_FILE = "/Users/user/beijingmap.osm"

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def process_map(file_in, validate):   

    #打开CSV文件
    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
        codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:
    
    #将表头分别写入CSV文件
        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()
       
    #将内容写入CSV文件
        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

if __name__ == '__main__':
    process_map(OSM_FILE , validate=False)